In [2]:
import torch
from torch import nn
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt


In [3]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [4]:
batch_size = 64

training_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

class ConvNet(nn.Module):
    def __init__(self) -> None:
        super(ConvNet, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
        self.flatten = nn.Flatten()

    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = ConvNet().to(device)
print(model)

Using cpu device
ConvNet(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
)


In [13]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [14]:
def train(data_loader, model, loss_fn, optimizer):
    size = len(data_loader.dataset)
    model.train()
    for batch, (X, y) in enumerate(data_loader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [15]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [16]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(training_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.283977  [    0/60000]
loss: 2.273722  [ 6400/60000]
loss: 2.252220  [12800/60000]
loss: 2.252122  [19200/60000]
loss: 2.231174  [25600/60000]
loss: 2.205641  [32000/60000]
loss: 2.207043  [38400/60000]
loss: 2.170106  [44800/60000]
loss: 2.161466  [51200/60000]
loss: 2.136252  [57600/60000]
Test Error: 
 Accuracy: 52.4%, Avg loss: 2.120972 

Epoch 2
-------------------------------
loss: 2.122272  [    0/60000]
loss: 2.113583  [ 6400/60000]
loss: 2.042628  [12800/60000]
loss: 2.070170  [19200/60000]
loss: 2.010868  [25600/60000]
loss: 1.950234  [32000/60000]
loss: 1.970281  [38400/60000]
loss: 1.885658  [44800/60000]
loss: 1.890091  [51200/60000]
loss: 1.819661  [57600/60000]
Test Error: 
 Accuracy: 61.5%, Avg loss: 1.806454 

Epoch 3
-------------------------------
loss: 1.835524  [    0/60000]
loss: 1.807700  [ 6400/60000]
loss: 1.670291  [12800/60000]
loss: 1.726714  [19200/60000]
loss: 1.607403  [25600/60000]
loss: 1.568776  [32000/600

In [52]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [53]:
new_model = ConvNet()
new_model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [55]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"


<!-- new_model = Conv -->